# Homework 5

In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


## Download Data

In [2]:
df = pd.read_csv('train.csv')
print(df.head())
print(df.info())


   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [3]:
df_test = pd.read_csv('test.csv')
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preparation

### Exporting first 10 rows as images

In [4]:
i = 0
with open('train.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header row if exists
    
    for data in csv_reader:
        # First column is label (0-9)
        label = int(data[0])
        
        # Remaining columns are pixels (0-255)
        pixels = data[1:]  # Skip first column
        pixels = np.array(pixels, dtype='uint8')
        
        # Reshape to 28x28
        pixels = pixels.reshape((28, 28))
        
        # Save image
        plt.imshow(pixels, cmap='pink')
        plt.title(f'Label: {label}')
        plt.savefig(f'image_{i}_label_{label}.png')
        plt.close()
        
        i += 1
        
        if i >= 10:  # Limit to first 10 images for testing
            break

### See if there are any missing values

In [5]:
print("\nMissing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Missing Values:
Series([], dtype: int64)


### Split data into $X$ and $y$

In [6]:
X = df.drop('label', axis=1)
y = df['label']

### Normalize pixel values

In [7]:
# for training set
X = X / 255

#for test set
df_test = df_test / 255


## Build and train model

### Train test split

In [8]:
# Split data for a train and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Fitting

In [9]:
# initialize the model
mlp = MLPClassifier(activation='logistic') 

# fit the model
mlp.fit(X_train, y_train)

# make predictions
y_pred = mlp.predict(X_test)

# calculate accuracy
mlp.score(X_test, y_test)

0.9716666666666667

### Hyperparameter tuning

In [10]:
# set the hyperparameter distributions
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'alpha': uniform(0.0001, 0.01),
}

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_dist,
    n_iter=100,
    cv=3,
    random_state=42,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Perform Random Search Cross-Validation
random_search.fit(X_train, y_train)

# Get the best parameters from RandomSearchCV
best_params = random_search.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END alpha=0.001934347898661638, hidden_layer_sizes=(50,); total time= 3.2min
[CV] END alpha=0.001934347898661638, hidden_layer_sizes=(50,); total time= 3.5min
[CV] END alpha=0.0038454011884736248, hidden_layer_sizes=(50,); total time= 3.6min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END alpha=0.0038454011884736248, hidden_layer_sizes=(50,); total time= 3.8min
[CV] END alpha=0.001934347898661638, hidden_layer_sizes=(50,); total time= 3.8min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END alpha=0.0038454011884736248, hidden_layer_sizes=(50,); total time= 3.8min
[CV] END alpha=0.006068501579464871, hidden_layer_sizes=(100,); total time= 4.1min
[CV] END alpha=0.006068501579464871, hidden_layer_sizes=(100,); total time= 5.0min
[CV] END alpha=0.004692488919658672, hidden_layer_sizes=(50,); total time= 7.5min


...

[CV] END alpha=0.0037965445606140446, hidden_layer_sizes=(50,); total time=11.0min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END alpha=0.0073821634861185965, hidden_layer_sizes=(50,); total time=10.6min
[CV] END alpha=0.0073821634861185965, hidden_layer_sizes=(50,); total time=26.1min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END alpha=0.0073821634861185965, hidden_layer_sizes=(50,); total time=26.1min
[CV] END alpha=0.0064230583059357955, hidden_layer_sizes=(100,); total time=25.6min
[CV] END alpha=0.0064230583059357955, hidden_layer_sizes=(100,); total time=26.0min
[CV] END alpha=0.0064230583059357955, hidden_layer_sizes=(100,); total time=56.8min
[CV] END alpha=0.004088244424445531, hidden_layer_sizes=(150,); total time=53.7min
[CV] END alpha=0.001002897700544083, hidden_layer_sizes=(50,); total time=36.7min
[CV] END alpha=0.004088244424445531, hidden_layer_sizes=(150,); total time=58.4min
[CV] END alpha=0.001002897700544083, hidden_layer_sizes=(50,); total time=37.0min
[CV] END alpha=0.004088244424445531, hidden_layer_sizes=(150,); total time=56.1min
[CV] END alpha=0.001002897700544083, hidden_layer_sizes=(50,); total time=37.8min
[CV] END alpha=0.0016071754396542947, hidden_layer_sizes=(150,); total time=40.3min
[CV] END alpha=0.0019651851039985424, hidden_layer_sizes=(50,); total time= 7.8min
[CV

In [13]:
best_params
best_model = mlp.set_params(**best_params)
best_model.fit(X_train, y_train)
best_accuracy = best_model.score(X_test, y_test)


In [14]:
best_accuracy

0.9745238095238096

In [ ]:
# set the hyperparameter distributions
param_grid = {
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
}


# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(estimator=best_model, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs = -1, verbose=2)
grid_search.fit(X, y)


# Get the best parameters from RandomSearchCV
best_params_3 = grid_search.best_params_


Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] END .......................activation=tanh, solver=adam; total time= 2.5min
[CV] END .......................activation=tanh, solver=adam; total time= 3.4min
[CV] END ...................activation=logistic, solver=adam; total time= 4.2min
[CV] END ...................activation=logistic, solver=adam; total time= 5.6min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END ........................activation=tanh, solver=sgd; total time= 6.0min
[CV] END ........................activation=tanh, solver=sgd; total time= 6.0min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END ....................activation=logistic, solver=sgd; total time= 6.1min


...

[CV] END ....................activation=logistic, solver=sgd; total time= 6.2min
[CV] END .......................activation=relu, solver=adam; total time= 2.1min
[CV] END .......................activation=relu, solver=adam; total time= 1.5min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END ........................activation=relu, solver=sgd; total time= 4.8min


/Users/blazingsun88/miniconda3/envs/ml/lib/python3.14/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END ........................activation=relu, solver=sgd; total time= 4.2min


In [22]:
best_params
best_model2 = best_model.set_params(**best_params)
best_model2.fit(X_train, y_train)
best_accuracy_2 = best_model2.score(X_test, y_test)



In [23]:
best_accuracy_2

0.9770238095238095

### Predicting

In [24]:
predictions = best_model2.predict(df_test)

predictions_df = pd.DataFrame({
    'ImageId': df_test.index + 1,  # use row indexes and start from 1 (since it is like this in the sample submission)
    'Label': predictions  # Predictions from the model
})


predictions_df.to_csv('second_tuning.csv', index=False)




# Discussion



I had an extremely hard time with the hyperparameter-tuning. During the last homework, I had used randomized search with 20 iteration and 5 cross-validations, and I found out that my model had a lower accuracy than the one with no hyperpparameter-tuning, therefore, this time I wanted to be extensive with 100 iterations and 3 cross-validations. However, my code took a scarily long time to finish running and I don't really see why it should have taken this long (it took 880 minutes and 22 seconds). My thoughts for why it shouldn't have taken this long are the following:
1.  If the default value of 'hidden_layer_sizes' is 100, me setting three values : (50,),(100,), and (150,) should not have been a problem.
2.  Since I had only 2 hyperparameters (hidden_layer_sizes and alpha), it shouldn't have taken this long. (I had tried setting up around 4 hyperparameters to tune but stopped at around 600 minutes)

I decided to train hidden_layer_sizes and alpha because I intuitively felt that they could affect the model the most (hidden_layer_sizes to make the model accurate and alpha to control overfitting).

Furthermore, I realized too late that I shouldn't have used the training set after splitting the data into train,test to perform the hyperparameter-tuning but I would've ran out of time if I redid it.
